In [1]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from obspy.core.event import read_events
from obspy.io.nordic.core import read_nordic
import os
import pandas as pd

/home/daniel/anaconda2/envs/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
cat = read_events("sfiles/20-0856-19L.S201802", format="NORDIC", encoding='latin-1')

event = cat.events[0]
origin = event.preferred_origin()
print(origin)

Origin
	   resource_id: ResourceIdentifier(id="smi:local/6488e36e-ba23-491a-a5f9-6cc87c581081")
	          time: UTCDateTime(2018, 2, 20, 8, 56, 19, 800000)
	     longitude: -74.38
	      latitude: 5.613
	         depth: 80100.0
	       quality: OriginQuality(standard_error=1.5, azimuthal_gap=41.0)
	 creation_info: CreationInfo(agency_id='RSN')
	          ---------
	      comments: 1 Elements
	      arrivals: 34 Elements


/home/daniel/rutinas/obspy/obspy/io/nordic/core.py:124: UserWarning: Lines of type I have not been implemented yet, please submit a development request
  "please submit a development request" % line_id)
/home/daniel/rutinas/obspy/obspy/io/nordic/core.py:600: UserWarning: Hypocentral Error estimates found, but not converted.
  warnings.warn("Hypocentral Error estimates found, but not converted.")
/home/daniel/rutinas/obspy/obspy/io/nordic/core.py:756: UserWarning: AIN: 155.0 in header, currently unsupported
  warnings.warn('AIN: %s in header, currently unsupported' % ain)
/home/daniel/rutinas/obspy/obspy/io/nordic/core.py:756: UserWarning: AIN: None in header, currently unsupported
  warnings.warn('AIN: %s in header, currently unsupported' % ain)
/home/daniel/rutinas/obspy/obspy/io/nordic/core.py:756: UserWarning: AIN: 129.0 in header, currently unsupported
  warnings.warn('AIN: %s in header, currently unsupported' % ain)
/home/daniel/rutinas/obspy/obspy/io/nordic/core.py:756: UserWarni

In [3]:
sta = pd.read_csv("stations.csv", error_bad_lines=False,
                 names=["net", "name", "lat", "lon", "z", "cod", "country"], header=None)
sta.head(15)

,net,name,lat,lon,z,cod,country
0,BR,BOAV,2.3953,-60.5225,114.0,,RR
1,BR,CZSB,-7.7299,-72.7049,196.0,,AC
2,BR,SGCB,-0.1223,-67.0306,70.0,,AM
3,BR,TBTG,-4.1868,-69.9090,91.0,,AM
4,CM,ACH1,3.9422,-73.6711,491.0,00,Colombia
5,CM,ACH2,3.9445,-73.7031,502.0,00,Colombia
6,CM,ACH3,3.9199,-73.6809,466.0,00,Colombia
7,CM,ACH4,3.9650,-73.6621,438.0,00,Colombia
8,CM,ACH5,3.8978,-73.5794,362.0,00,Colombia
9,CM,ACH6,3.8974,-73.7812,566.0,00,Colombia


In [9]:
string_cols = ["name", "net", "cod", "country"]



,net,name,lat,lon,z,cod,country
151,CM,PTA,7.1472,-77.8083,78.0,00,Colombia
152,CM,PTA,7.1472,-77.8083,78.0,10,Colombia
153,CM,PTA,7.1472,-77.8083,78.0,40,Colombia


In [53]:
df_sta = sta[sta["name"]=="URE"]
#type(df_sta[sta["cod"] == "20"]["cod"].any())
df_sta["net"].values[0]

'CM'

In [73]:
sta = pd.read_csv("stations.csv", error_bad_lines=False,
                   names=["net", "name", "lat", "lon", "z",
                          "cod", "country"], header=None)

def get_loc_cha(station, sta):
    """ Devuelve el codigo de localización, el canal y la red de una
    estación dada. Los datos se toman del dataframe de pandas 
    alimentado por el fichero statios.csv
    
    :type station: str
    :param station: Nombre de la estación
    :type sta: Pandas Dataframe
    :param: Dataframe de pandas con la información de las estaciones 
    que adquiere el SGC. 
    """
    
    loc_cod = 0
    
    columns = ["name", "net", "cod"]
    for col in columns:
        sta[col] = sta[col].apply(lambda x: x.strip())


    
    df_sta = sta[sta["name"]==station]
    
    # se prueba con los codigos de localizacion posibles priorizando
    # los banda ancha (00)
    codes = ["00", "20", "10", "40"]
    for cod in codes:
        loc_cod = df_sta[sta["cod"] == cod]["cod"].any()
        #print(loc_cod)
        if loc_cod != 0: break
    
    cha = {"00":"HH", "20":"EH", "10":"HN", "40":"HL", 0:""}
    
    #print(station)
    net = df_sta["net"].values[0]
    
    return str(loc_cod), cha[loc_cod], net
    
    

In [57]:
get_loc_cha("OTAV", sta)

/home/daniel/anaconda2/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


('00', 'HH', 'IU')

In [6]:
pick = event.picks[0]
pick.waveform_id.network_code = "CM"
pick.waveform_id.channel_code = "00"
print(type(pick))
pick.waveform_id

<class 'obspy.core.event.origin.Pick'>


WaveformStreamID(network_code='CM', station_code='SPBC', channel_code='00')

In [72]:
from obspy.core.event import Catalog
cat = Catalog()

folder = "sfiles/"
for sfile in os.listdir(folder):
    print(sfile)
    event = read_events(folder+sfile, format="NORDIC", encoding='latin-1')
    
    for pick in event.events[0].picks:
        
        station = pick.waveform_id.station_code
        
        if station[:4] != "SPEC":
            loc, cha, net = get_loc_cha(station, sta)

            pick.waveform_id.location_code = loc
            pick.waveform_id.channel_code = cha
            pick.waveform_id.network_code = net
        
        
    cat.append(event.events[0])

21-1918-32L.S201712
GARC
GARC
GARC
MARO
MARO
MARO
CONO
CONO
CONO
FLO2
FLO2
FLO2
SOTO
SOTO
ORTC
ORTC
ORTC
23-1800-21L.S201802
GARC


/home/daniel/anaconda2/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


GARC
GARC
MARO
MARO
MARO
PRA
PRA
PRA
FLO2
FLO2
FLO2
ORTC
ORTC
ORTC
CONO
CONO
CONO
SOTO
SOTO
SOTO
ANIL
ANIL
ANIL
25-0009-48L.S201712
BAR2
BAR2
BAR2
PAM
PAM
PAM
PTB
PTB
PTB
TAM
TAM
TAM
OCA
OCA
OCA
SPBC
SPBC
SPBC
YPLC
YPLC
YPLC
ZAR
ZAR
ZAR
NOR
ROSC
ROSC
ROSC
URE
URE
DRL03
DRL03
GUY2C
GUY2C
GUY2C
VIL
VIL
PTGC
PTGC
PTGC
LL6C
LL6C
LL6C
ELA
ELA
ANIL
PRA
PRA
PRA
ORTC
ORTC
ORTC
SJC
SJC
SJC
23-1645-57L.S201712
CONO
CONO
CONO
GARC
GARC
GARC
SOTO
SOTO
MARO
MARO
MARO
27-2115-32L.S201712
PAM
PAM
PAM
BAR2
BAR2
BAR2
ZAR
ZAR
ZAR
PTB
PTB
PTB
25-1536-32L.S201712
BAR2
BAR2
BAR2
PAM
PAM
PAM
PTB
PTB
PTB
OCA
OCA
OCA
SPBC
SPBC
SPBC
ZAR
ZAR
ZAR
URE
URE
GUY2C
GUY2C
GUY2C
DRL02
DRL02
PRA
PRA
PRA
ORTC
ORTC
ORTC
SJC
SJC
SJC
20-0226-18L.S201802
CAP2
CAP2
UPD2
UPD2
APAC
APAC
PTA
PTA
PTA
URE
URE
URE
CBOC
CBOC
CBOC
24-1015-43L.S201802
TUM
TUM
TUM
PAC1
PAC1
GR1C
GR1C
GCUF
GCUF
GCUF
BBAC
BBAC
BBAC
SOTO
SOTO
SOTO
POP2
POP2
POP2
CONO
CONO
CONO
MAL
MAL
MAL
JAMC
26-2216-02L.S201802
NOR
NOR
NOR
PTB
PTB
PTB
HEL
HEL
HEL
SPBC
S

NOR
NOR
NOR
GUY2C
GUY2C
GUY2C
CHI
CHI
CHI
ORTC
ORTC
ORTC
21-0852-01L.S201802
FLO2
FLO2
GARC
GARC
GARC
CONO
CONO
CONO
SOTO
SOTO
SOTO
POP2
21-0536-25L.S201712
POP2
POP2
POP2
BBAC
BBAC
BBAC
SOTO
SOTO
CONO
JAMC
23-1113-12L.S201712
BAR2
BAR2
BAR2
PAM
PAM
PAM
PTB
PTB
PTB
SPBC
SPBC
SPBC
ZAR
24-1043-14L.S201802
PAM
PAM
PAM
OCA
OCA
BAR2
BAR2
RUS
RUS
PTB
PTB
PTB
DRL02
DRL02
URE
URE
URE
NOR
NOR
HEL
HEL
HEL
SJC
SJC
SJC
CHI
CHI
CHI
GUY2C
GUY2C
GUY2C
21-0836-20L.S201802
BAR2
BAR2
BAR2
PAM
PAM
PAM
PTB
PTB
PTB
OCA
OCA
OCA
TAM
TAM
SPBC
SPBC
SPBC
YPLC
YPLC
YPLC
NOR
NOR
NOR
CHI
CHI
CHI
GUY2C
GUY2C
GUY2C
VIL
VIL
PTGC
PTGC
PTGC
DRL02
DRL02
LL6C
SDV
PRA
PRA
PRA
SJC
SJC
SJC
21-0219-10L.S201712
CRU
CRU
BBAC
BBAC
BBAC
SOTO
SOTO
GCUF
27-0707-44L.S201712
CBOC
CBOC
CBOC
DBB
DBB
DBB
PAL
PAL
PAL
GUY2C
GUY2C
GUY2C
URE
YOT
24-0945-00L.S201712
BAR2
BAR2
PTB
PTB
PTB
SPBC
ZAR
23-1508-08L.S201802
BAR2
BAR2
BAR2
PAM
PAM
PAM
RUS
RUS
RUS
PTB
PTB
PTB
OCA
OCA
OCA
TAM
TAM
TAM
SPBC
SPBC
SPBC
YPLC
YPLC
YPLC
NOR
NOR
NOR
CHI
CHI
C

PRA
ORTC
ORTC
ORTC
SJC
SJC
SJC
PAL
PAL
PAL
VALLC
VALLC
LCBC
LCBC
CRJC
CRJC
CRJC
YOT
YOT
YOT
SMAR
SMAR
SMAR
CAP2
CAP2
CAP2
PTA
PTA
PTA
BET
BET
BET
MACC
MACC
MARO
MARO
MARO
JAMC
JAMC
GARC
CONO
CONO
CONO
POP2
POP2
FLO2
FLO2
FLO2
SOTO
SOTO
BCIP
BCIP
27-1858-56L.S201802
PAM
PAM
PAM
OCA
OCA
BAR2
BAR2
BAR2
RUS
RUS
RUS
TAM
TAM
TAM
PTB
PTB
PTB
DRL03
DRL03
LL6C
YPLC
YPLC
YPLC
URE
URE
NOR
NOR
NOR
HEL
HEL
HEL
SJC
SJC
SJC
DBB
DBB
DBB
GUY2C
GUY2C
GUY2C
CBOC
CBOC
CBOC
CRJC
CRJC
CRJC
APAC
APAC
APAC
ELA
ELA
ELA
PAL
PAL
ORTC
22-1150-46L.S201802
BAR2
BAR2
BAR2
PAM
PAM
PAM
RUS
RUS
RUS
PTB
PTB
PTB
OCA
OCA
OCA
29-0751-58L.S201712
BAR2
BAR2
BAR2
PAM
PAM
PAM
PTB
PTB
PTB
OCA
SPBC
SPBC
SPBC
ZAR
ZAR
ZAR
GUY2C
GUY2C
GUY2C
23-1314-06L.S201802
DRL03
DRL03
LL6C
DRL02
LL6C
LL6C
DRL02
SJC
SJC
SJC
27-1709-07L.S201802
BAR2
BAR2
PAM
PAM
PAM
RUS
RUS
RUS
PTB
PTB
PTB
SPBC
SPBC
SPBC
NOR
GUY2C
GUY2C
GUY2C
DRL03
DRL03
20-1222-28L.S201712
ZAR
ZAR
ZAR
PTB
PTB
PTB
BAR2
BAR2
BAR2
OCA
PAM
PAM
PAM
URE
URE
URE
SPBC
SPBC
SPBC
GUY2C
GU

...

BAR2
BAR2
BAR2
PAM
PAM
PAM
RUS
RUS
RUS
PTB
PTB
PTB
TAM
TAM
TAM
OCA
OCA
OCA
SPBC
SPBC
SPBC
YPLC
YPLC
NOR
NOR
NOR
CHI
CHI
CHI
ROSC
ROSC
ROSC
HEL
HEL
HEL
GUY2C
GUY2C
GUY2C
DRL02
CBOC
CBOC
CBOC
PRA
PRA
PRA
ORTC
ORTC
ORTC
SJC
SJC
SJC
25-0911-48L.S201712
CBOC
CBOC
CBOC
PAL
PAL
PAL
GUY2C
GUY2C
GUY2C
ELA
PTB
26-0913-33L.S201712
BAR2
BAR2
BAR2
PAM
PAM
PAM
PTB
PTB
PTB
TAM
TAM
TAM
OCA
OCA
OCA
SPBC
SPBC
SPBC
YPLC
YPLC
YPLC
ZAR
ZAR
ZAR
24-1355-56L.S201712
CCALA
CCALA
ANIL
ANIL
ELA
ELA
ELA
ORTC
PAL
PAL
PAL
GUY2C
GUY2C
GUY2C
YOT
YOT
YOT
PRA
PRA
PRA
26-2153-35L.S201712
BAR2
BAR2
BAR2
PAM
PAM
PAM
PTB
PTB
ZAR
20-1209-31L.S201712
BAR2
BAR2
BAR2
PAM
PAM
PAM
PTB
PTB
PTB
OCA
OCA
OCA
SPBC
SPBC
SPBC
ZAR
ZAR
ZAR
GUY2C
GUY2C
22-2147-28L.S201712
YOT
YOT
YOT
MAL
MAL
PAL
PAL
PAL
JAMC
JAMC
JAMC
27-0535-23L.S201712
DBB
DBB
DBB
URE
URE
URE
ZAR
ZAR
ZAR
CBOC
CBOC
CBOC
PTB
PTB
PTB
21-0918-02L.S201802
BAR2
BAR2
BAR2
PAM
PAM
PAM
PTB
PTB
PTB
SPBC
SPBC
SPBC
NOR
NOR
NOR
CHI
CHI
CHI
GUY2C
GUY2C
GUY2C
SJC
SJC
23-1519-18L.S2017

PTGC
LL6C
LL6C
LL6C
DBB
DBB
DBB
ARGC
ARGC
ARGC
SDV
SDV
SDV
APAC
APAC
APAC
PRA
PRA
PRA
ORTC
ORTC
ORTC
SJC
SJC
SJC
PAL
PAL
PAL
CRJC
CRJC
CRJC
YOT
YOT
SMAR
SMAR
SMAR
CAP2
CAP2
CAP2
PTA
PTA
BET
BET
BET
MACC
MACC
MACC
JAMC
JAMC
JAMC
UPD2
UPD2
UPD2
POP2
POP2
FLO2
FLO2
FLO2
BBAC
BBAC
BBAC
GCUF
22-0748-43L.S201712
BAR2
BAR2
BAR2
PAM
PAM
PAM
PTB
PTB
PTB
OCA
OCA
OCA
SPBC
SPBC
SPBC
ZAR
ZAR
GUY2C
22-0933-30L.S201802
BAR2
BAR2
BAR2
PAM
PAM
PAM
RUS
RUS
RUS
SPBC
SPBC
25-0949-35L.S201712
BAR2
BAR2
BAR2
PAM
PAM
PAM
PTB
PTB
PTB
TAM
TAM
TAM
OCA
OCA
OCA
SPBC
SPBC
SPBC
YPLC
YPLC
YPLC
ZAR
ZAR
ZAR
ROSC
ROSC
ROSC
CVER
CVER
GUY2C
GUY2C
GUY2C
PTGC
PTGC
ELA
CBOC
CBOC
CBOC
PRA
PRA
PRA
25-0640-07L.S201712
BAR2
BAR2
BAR2
PAM
PAM
PAM
PTB
PTB
PTB
TAM
TAM
OCA
OCA
OCA
SPBC
ZAR
ZAR
ZAR
DRL03
GUY2C
GUY2C
GUY2C
CBOC
CBOC
CBOC
SJC
23-1127-48L.S201712
BAR2
BAR2
BAR2
PAM
PAM
PAM
PTB
PTB
PTB
OCA
SPBC
SPBC
SPBC
ZAR
25-0116-13L.S201802
BAR2
BAR2
PAM
PAM
PAM
RUS
RUS
RUS
PTB
PTB
PTB
OCA
OCA
SPBC
SPBC
NOR
NOR
CHI
CHI
CHI
DRL03
DRL

PAM
RUS
RUS
RUS
PTB
PTB
PTB
OCA
OCA
OCA
27-0410-34L.S201712
BAR2
BAR2
BAR2
PTB
PTB
PTB
SPBC
SPBC
SPBC
OCA
ZAR
ZAR
ZAR
GUY2C
GUY2C
GUY2C
27-1554-37L.S201712
BAR2
BAR2
BAR2
PAM
PAM
PAM
PTB
PTB
PTB
TAM
TAM
SPBC
SPBC
YPLC
YPLC
YPLC
ZAR
ZAR
ZAR
GUY2C
GUY2C
GUY2C
PRA
PRA
PRA
ORTC
ORTC
ORTC
22-1732-50L.S201802
BAR2
BAR2
BAR2
PAM
PAM
PAM
RUS
RUS
RUS
PTB
PTB
PTB
TAM
TAM
TAM
OCA
OCA
OCA
SPBC
SPBC
SPBC
YPLC
YPLC
YPLC
NOR
NOR
NOR
CHI
CHI
CHI
ROSC
ROSC
ROSC
HEL
HEL
HEL
CVER
CVER
CVER
URE
URE
URE
DRL03
DRL03
GUY2C
GUY2C
GUY2C
VIL
VIL
VIL
PTGC
PTGC
PTGC
ELA
ELA
ELA
DBB
DBB
DBB
ARGC
ARGC
ARGC
SDV
SDV
SDV
ANIL
ANIL
ANIL
PRA
PRA
PRA
ORTC
ORTC
ORTC
SJC
SJC
SJC
CRJC
CRJC
CRJC
SMAR
SMAR
SMAR
20-0413-15L.S201712
SPBC
SPBC
SPBC
PTB
PTB
PTB
BAR2
BAR2
BAR2
YPLC
YPLC
YPLC
GUY2C
GUY2C
GUY2C
PRA
PRA
PRA
ORTC
ORTC
ORTC
20-1755-18L.S201802
BET
BET
CONO
CONO
BBAC
BBAC
CRU
BAR2
PAM
PAM
PAM
TAM
TAM
PTB
PTB
PTB
TAM
OCA
OCA
OCA
SPBC
SPBC
SPBC
YPLC
YPLC
YPLC
NOR
NOR
NOR
CHI
ROSC
ROSC
ROSC
CVER
HEL
HEL
HEL
URE
URE
URE
DRL

PAM
RUS
RUS
RUS
PTB
PTB
PTB
SPBC
SPBC
SPBC
NOR
NOR
NOR
CHI
CHI
CHI
ROSC
26-1119-06L.S201802
VIL
VIL
VIL
CVER
CVER
CVER
ORTC
ORTC
ORTC
IBA1C
IBA1C
IBA1C
SPBC
SPBC
SPBC
NOR
NOR
NOR
CCALA
CCALA
NIZA
NIZA
GUY2C
GUY2C
GUY2C
PTGC
PTGC
PTGC
RUS
RUS
RUS
MACC
MACC
MACC
YPLC
YPLC
YPLC
BET
BET
BET
YOT
YOT
YOT
PAL
PAL
PAL
PTB
PTB
PTB
GARC
GARC
BAR2
BAR2
BAR2
HEL
HEL
HEL
JAMC
JAMC
JAMC
FLO2
FLO2
FLO2
POP2
POP2
POP2
TAM
MAL
MAL
MAL
PAM
PAM
PAM
DBB
BBAC
BBAC
BBAC
URE
URE
URE
OCA
OCA
OCA
SDV
SDV
SDV
SJC
SJC
SJC
28-2023-30L.S201802
FLO2
FLO2
FLO2
SOTO
CONO
CONO
CONO
GARC
GARC
SOTO
SOTO
25-2039-42L.S201802
YPLC
YPLC
YPLC
TAM
TAM
TAM
RUS
RUS
RUS
BAR2
BAR2
PAM
PAM
PAM
PTGC
PTGC
SPBC
VIL
25-0656-19L.S201802
PAM
BAR2
BAR2
BAR2
RUS
RUS
RUS
PTB
SPBC
20-0327-27L.S201802
BAR2
BAR2
BAR2
PAM
PAM
PAM
PTB
PTB
PTB
OCA
OCA
OCA
SPBC
SPBC
SPBC
CHI
CHI
CHI
DRL03
DRL03
PTGC
PTGC
SJC
SJC
SJC
27-1221-32L.S201802
IBA1C
IBA1C
CCALA
CCALA
ORTC
ORTC
ORTC
GUY2C
GUY2C
GUY2C
PAL
PAL
PAL
YOT
YOT
NOR
NOR
NOR
CVER
CVER
CVER
HEL
HEL


BAR2
PAM
PAM
PAM
RUS
RUS
RUS
PTB
PTB
PTB
OCA
OCA
SPBC
SPBC
SPBC
NOR
NOR
NOR
CHI
CHI
CHI
27-2056-46L.S201802
PAM
PAM
PAM
RUS
RUS
RUS
PTB
PTB
PTB
SPBC
NOR
21-0804-00L.S201712
BAR2
BAR2
BAR2
PAM
PAM
PAM
PTB
PTB
PTB
OCA
OCA
OCA
ZAR
ZAR
ZAR
URE
URE
URE
GUY2C
GUY2C
CBOC
CBOC
CBOC
PRA
PRA
PRA
SJC
SJC
SJC
25-2113-15L.S201712
BAR2
BAR2
BAR2
PAM
PAM
PAM
PTB
PTB
PTB
OCA
OCA
SPBC
SPBC
SPBC
YPLC
ZAR
ZAR
URE
GUY2C
GUY2C
GUY2C
SDV
SDV
SDV
PRA
PRA
PRA
25-1028-21L.S201712
ANIL
ANIL
ANIL
GUY2C
GUY2C
GUY2C
CCALA
CCALA
PRA
PRA
PRA
24-0548-12L.S201712
OCA
OCA
OCA
PAM
PAM
PAM
BAR2
BAR2
BAR2
ZAR
ZAR
ZAR
PTB
PTB
PTB
26-1302-01L.S201802
CBOC
CBOC
CBOC
HEL
HEL
HEL
PAL
PAL
PAL
DBB
DBB
NOR
NOR
NOR
PIZC
CCALA
YOT
YOT
YOT
PTB
PTB
PTB
URE
URE
URE
MAL
MAL
ORTC
ORTC
ORTC
JAMC
JAMC
RUS
RUS
RUS
POP2
POP2
POP2
PAM
PAM
PAM
27-1501-57L.S201802
VIL
VIL
VIL
CVER
CVER
CVER
ROSC
ROSC
ROSC
ORTC
ORTC
ORTC
SPBC
SPBC
SPBC
CCALA
CCALA
NOR
NOR
NOR
GUY2C
GUY2C
GUY2C
MACC
MACC
MACC
RUS
RUS
RUS
YPLC
YPLC
YPLC
YOT
YOT
YOT
PTB
PTB
PTB
CO

In [58]:
#cat.plot(projection="local")
#cat.filter("time > 2018-02-01T00:16",
           #"time < 2018-02-01T00:18")
    

In [74]:
cat.write("catalogo_diciembre_2017.xml", format="SC3ML")